In [ ]:
import polars as pl

df_brasileirao = pl.read_parquet(fr'C:\Users\Thiago_SL\projeto-backend\Football-Analytics-Platform-Back-End\brasileirao.parquet')

df_campbras = pl.read_parquet(fr'C:\Users\Thiago_SL\projeto-backend\Football-Analytics-Platform-Back-End\campbras.parquet')

df_campbras

df_brasileirao = df_brasileirao.with_columns(pl.when(pl.col("team") == 'SaoPaulo').then("São Paulo").when(pl.col("team") == "Atletico-MG").then("Atlético-MG").when(pl.col("team") == "America-MG").then("América-MG").when(pl.col("team") == 'Goias').then("Goiás").when(pl.col("team") == 'RBBragantino').then("Bragantino").when(pl.col("team") == 'Cuiaba').then('Cuiabá').when(pl.col("team") == 'Ceara').then('Ceará').when(pl.col("team") == "Atletico-GO").then("Atlético-GO").when(pl.col("team") == "Avai").then("Avaí").otherwise(pl.col("team")).alias("team"))
df_brasileirao_tabela = df_brasileirao.filter(pl.col("season") == 2022)

df_filtrado = df_campbras.filter(pl.col("year") == 2022)
# print(df_filtrado)

gols_mandante = (df_filtrado.groupby("hometeam").agg(pl.col("goalsht").sum().alias("goalsht")).rename({"hometeam":"team"}))            

goals_visitantes = (df_filtrado.groupby("visitingteam").agg(pl.col("goalsvt").sum().alias("goalsvt")).rename({"visitingteam":"team"}))  

gols_totais = (gols_mandante.join(goals_visitantes, on="team", how="outer").fill_null(0).with_columns(pl.col("goalsht") + pl.col("goalsvt")))

df_brasileirao_tabela = df_brasileirao_tabela.join(gols_totais, on="team", how="left")   
df_brasileirao_tabela
# df_campbras.filter(pl.col("year")==2022)

season,place,acronym,team,points,played,won,draw,loss,goalsht,goalsvt
i64,i64,str,str,i64,i64,i64,i64,i64,i64,i64
2022,1,"""PAL""","""Palmeiras""",81,38,23,12,3,66,27
2022,2,"""INT""","""Internacional""",73,38,20,13,5,58,18
2022,3,"""FLU""","""Fluminense""",70,38,21,7,10,63,24
2022,4,"""COR""","""Corinthians""",65,38,18,11,9,44,20
2022,5,"""FLA""","""Flamengo""",62,38,18,8,12,60,23
2022,6,"""CAP""","""Athletico-PR""",58,38,16,10,12,48,17
2022,7,"""CAM""","""Atlético-MG""",58,38,15,13,10,45,20
2022,8,"""FOR""","""Fortaleza""",55,38,15,10,13,46,22
2022,9,"""SAO""","""São Paulo""",54,38,13,15,10,55,23
